### Initialize notebook variables

Configures everything that's needed for deployment. **Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

**Note:** If the specified infrastructure doesn't exist, the deployment process will automatically query for available infrastructures and allow you to select from existing options.

In [6]:
# Restart and reload the utils module
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 43  # Changed to test infrastructure creation option
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['general']

# 2) Service-defined parameters (please do not change these)
rg_name       = utils.get_infra_rg_name(deployment, index)
sample_folder = "general"
nb_helper     = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM])

# 3) Define the APIs and their operations and policies

# API 1
api1_get = GET_APIOperation('This is a GET for API 1')
api1_post = POST_APIOperation('This is a POST for API 1')
api1 = API('API1', 'API 1', '/api1', 'This is API 1', operations = [api1_get, api1_post], tags = tags)

# API 2
api2_post = POST_APIOperation('This is a POST for API 2')
api2 = API('API2', 'API 2', '/api2', 'This is API 2', operations = [api2_post], tags = tags)

# Request Headers
pol_request_headers_get = utils.read_policy_xml(REQUEST_HEADERS_XML_POLICY_PATH)
request_headers_get     = GET_APIOperation('Gets the request headers for the current request and returns them. Great for troubleshooting.', pol_request_headers_get)
request_headers         = API('requestheaders', 'Request Headers', '/request-headers', 'API for request headers', operations = [request_headers_get], tags = tags)

# APIs Array
apis: List[API] = [api1, api2, request_headers]

📄 Reading policy XML from  : C:\Dev\Azure-Samples\Apim-Samples\shared\apim-policies\request-headers.xml


### Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
# Check infrastructure availability and let user select or create
print("Checking infrastructure availability...")
print(f"Desired infrastructure: {deployment.value}")
print(f"Desired index: {index}")
print(f"Desired resource group: {rg_name}")

# Call the resource group existence check only once
rg_exists = utils.does_resource_group_exist(rg_name)
print(f"Resource group exists: {rg_exists}")

# If the desired infrastructure doesn't exist, use the interactive selection process
if not rg_exists:
    print(f"\nDesired infrastructure does not exist. Querying for available options...")
    
    # Check if we've already done infrastructure selection (prevent double execution)
    if 'infrastructure_selection_completed' not in globals():
        # Use the NotebookHelper's infrastructure selection process
        selected_deployment, selected_index = nb_helper._query_and_select_infrastructure()
        
        if selected_deployment is None:
            raise SystemExit(1)
        
        # Update the notebook helper with the selected infrastructure
        nb_helper.deployment = selected_deployment
        nb_helper.rg_name = utils.get_infra_rg_name(selected_deployment, selected_index)
        
        print(f"✅ Using infrastructure: {selected_deployment.value}")
        print(f"📦 Resource group: {nb_helper.rg_name}")
        
        # Update the variables for the rest of the notebook
        deployment = selected_deployment
        rg_name = nb_helper.rg_name
        index = selected_index
        
        # Verify the updates were applied correctly
        print(f"📝 Updated variables: deployment = {deployment.value}, index = {index}, rg_name = {rg_name}")
        
        # Mark infrastructure selection as completed
        infrastructure_selection_completed = True
    else:
        print("✅ Infrastructure selection already completed in this session")
    
else:
    print("✅ Desired infrastructure already exists, proceeding with sample deployment")

Checking infrastructure availability...
Desired infrastructure: simple-apim
Desired index: 43
Desired resource group: apim-infra-simple-apim-43
⚙️ az group show --name apim-infra-simple-apim-43 
Resource group exists: False

Desired infrastructure does not exist. Querying for available options...

👉🏽 Querying for available infrastructures... 
👉🏽 Found 6 existing infrastructure(s). You can select an existing or create a new one. 

     1. simple-apim (index: 6) - Resource Group: apim-infra-simple-apim-6
     2. simple-apim (index: 7) - Resource Group: apim-infra-simple-apim-7
     3. simple-apim (index: 8) - Resource Group: apim-infra-simple-apim-8
     4. simple-apim (index: 9) - Resource Group: apim-infra-simple-apim-9
     5. simple-apim (index: 42) - Resource Group: apim-infra-simple-apim-42
     6. simple-apim (index: 99) - Resource Group: apim-infra-simple-apim-99
     7. 🚀 Create new infrastructure: simple-apim (index: 43) - Resource Group: apim-infra-simple-apim-43

✅ Selected e

In [ ]:
# Deploy the sample APIs to the selected infrastructure
print(f"\n🚀 Deploying sample APIs to infrastructure: {deployment.value}")
print(f"📦 Resource group: {rg_name}")

# Build the bicep parameters
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]}
}

# Deploy directly to avoid duplicate logging issues
import os
import json
from pathlib import Path

# Get the current sample directory
sample_dir = Path.cwd() if Path.cwd().name == sample_folder else utils.find_project_root() / 'samples' / sample_folder
original_cwd = os.getcwd()

try:
    # Change to sample directory
    os.chdir(sample_dir)
    print(f"📁 Changed working directory to: {sample_dir}")
    
    # Prepare deployment parameters
    bicep_parameters_format = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": bicep_parameters
    }
    
    # Write the parameters file
    params_file_path = sample_dir / 'params.json'
    with open(params_file_path, 'w') as file:
        file.write(json.dumps(bicep_parameters_format))
    
    print(f"📝 Updated the bicep parameters file 'params.json'")
    
    # Run the deployment directly
    main_bicep_path = sample_dir / 'main.bicep'
    output = utils.run(
        f'az deployment group create --name {sample_folder} --resource-group {rg_name} --template-file "{main_bicep_path}" --parameters "{params_file_path}" --query "properties.outputs"',
        f"Sample deployment '{sample_folder}' succeeded", 
        f"Sample deployment '{sample_folder}' failed."
    )
    
    if output.success:
        # Extract deployment outputs for testing
        apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
        apim_apis = output.getJson('apiOutputs', 'APIs')

        print(f"✅ Sample deployment completed successfully!")
        print(f"🌐 APIM Gateway URL: {apim_gateway_url}")
        print(f"📋 APIs deployed: {len(apim_apis)}")
    else:
        print("❌ Sample deployment failed!")
        raise SystemExit(1)
        
finally:
    # Always restore the original working directory
    os.chdir(original_cwd)
    print(f"📁 Restored working directory to: {original_cwd}")

### Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
from apimrequests import ApimRequests
from apimtesting import ApimTesting

# Use the current deployment and resource group name (may have changed during infrastructure selection)
current_deployment = nb_helper.deployment
current_rg_name = nb_helper.rg_name

tests = ApimTesting("General Sample Tests", sample_folder, current_deployment)

api_subscription_key = apim_apis[2]['subscriptionPrimaryKey']

# Check if the infrastructure architecture deployment uses Azure Front Door.
utils.print_message('Checking if the infrastructure architecture deployment uses Azure Front Door.', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(current_deployment, current_rg_name)

if afd_endpoint_url:
    reqsAfd = ApimRequests(afd_endpoint_url, api_subscription_key)
    output = reqsAfd.singleGet('/request-headers', msg = 'Calling Request Headers API via via Azure Front Door. Expect 200.')
    tests.verify('Host:' in output, True)
else:
    # Issue a direct request to API Management
    reqsApim = ApimRequests(apim_gateway_url, api_subscription_key)
    output = reqsApim.singleGet('/request-headers', msg = 'Calling Request Headers API via API Management Gateway URL. Response codes 200 and 403 are both valid depending on the infrastructure used.')
    tests.verify('Host:' in output, True)

tests.print_summary()

utils.print_ok('All done!')